In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['CURL_CA_BUNDLE'] = ''

Базовый пайплайн из коробки

In [2]:
credentials = os.getenv('credentials')

In [3]:

llm = GigaChat(credentials=credentials, verify_ssl_certs=False)#auth_url = 'https://sm-auth-sd.prom-88-89-apps.ocp-geo.ocp.sigma.sbrf.ru/api/v2/oauth'

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя.
Используй при этом только информацию из контекста. Отвечай на русском языке.
Контекст: {context}
Вопрос: {input}
Ответ:''')
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    )

C:\Users\Grisha\AppData\Local\Temp\ipykernel_4292\2433700320.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(
c:\Users\Grisha\Desktop\RAGWORK\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Grisha\Desktop\RAGWORK\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers 

In [4]:
loader = PyPDFLoader('ai-hr.pdf')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [5]:
len(documents)

131

In [6]:
vectorstore = FAISS.from_documents(documents=documents, embedding=embedding)

In [9]:
! pip install pdfminer.six


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from langchain.document_loaders import PDFMinerLoader

In [11]:
import re

def clean_text_from_pdf(docum):
    for chunk in range(len(docum)):
        text = doc[chunk].page_content
        text = re.sub(r'http[s]?://\S+|www\.\S+', '', text)  
    # Убираем изображения, которые могут быть представлены как ссылки или текст
        text = re.sub(r'\[.*?\]\(.*?\)|(?:download(?:[^\n]*?))|(?:image(?:[^\n]*?))', '', text)
    # Убираем строки, представляющие собой таблицы (например, строка с символами "|")
        text = re.sub(r'(^|\n)\|.*?\|(\n|$)', '', text) # убрать строки с разделителем `|
    # Удаляем специальные символы, оставляя только буквы, цифры и пробелы
        text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', text)
    # Удаляем лишние пробелы и пробелы в начале и в конце строки
        text = re.sub(r'\s+', ' ', text).strip()
        docum[chunk].page_content = text
    return doc

In [12]:
loader = PDFMinerLoader('ai-hr.pdf',concatenate_pages=True)
doc = loader.load()
doc = clean_text_from_pdf(doc)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=300)
clean_data = text_splitter.split_documents(doc)


In [13]:
len(clean_data)

150

In [14]:

vectorstore_clean = FAISS.from_documents(documents=clean_data, embedding=embedding)

In [7]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [8]:
! pip install rank_bm25


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
retriever_clean = vectorstore_clean.as_retriever(search_kwargs={"k": 3}, search_type="mmr")

In [10]:
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 2

In [86]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_clean], weights=[0.75, 0.25])

In [13]:
retriever_d = vectorstore.as_retriever(search_kwargs={"k": 3}, search_type="mmr")

In [14]:
ensemble_retriever_d = EnsembleRetriever(retrievers=[bm25_retriever, retriever_d], weights=[0.01, 0.99])

In [15]:
sysprompt = ('''Твоя роль: Аналитик специализирующийся на быстром поиске информации в предоставленном контексте.
Краткая инструкция: Анализировать предложенный контекст и отвечать на вопросы, опираясь исключительно на информацию из этого контекста.
Если информации в документе нет или не хватает, то так и пиши: информации недостаточно. Всегда отвечай на русском языке.
Твоя цель: Обеспечить структурированный и развёрнутый ответ, полностью основанный на информации из предложенного контекста, без внесения внешних данных или предположений.
Результат: Ответ должен быть четким и точным, содержать только информацию из предложенного контекста.
Ожидается, что ответ будет логически обоснованным и последовательным. Если ответа на вопрос нет в контексте, скажи об этом пользователю
Ограничения: Отвечать можно только по контексту.

Пример:
Контекст: Борщ варится из куриных лапок. Велосипеды марки крути педали имеют три колеса.
Вопрос: Когда изобрели утюг?
Ответ: Информации нет в контексте.

Пример:
Контекст: Вася Пупкин изобрёл специальный банкомат в 2000 году. Банкомат нужен для выдачи пончиков.
Вопрос: Когда и для чего изобрели специальный банкомат?
Ответ: Банкомат был изобретён Васей Пупкиным в 2000 году и он нужен для выдачи пончиков.

Пример:
Контекст: Автомобильная марка Audi имеет тот же логотип, что и марка ВАЗ. Автомобильная марка ВАЗ славится своими газонокосилками, хоть и производит автомобили.
Вопрос: Чем известа автомобильная компания ВАЗ?
Ответ: Автомобильная компания ВАЗ производит автомобили и газонокосили. Газонокосилки компании ВАЗ очень известны.

Контекст: {context}
Ответ:'''
)  
    # """Твоя роль: Аналитик специализирующийся на быстром поиске информации в предоставленном контексте.
    # Краткая инструкция: Анализировать предложенный контекст и отвечать на вопросы, опираясь исключительно на информацию из этого контекста. 
    # Если информации в документе нет или не хватает, то так и пиши: информации недостаточно. Всегда отвечай на русском языке.
    # Твоя цель: Обеспечить структурированный и развёрнутый ответ, полностью основанный на информации из предложенного контекста, без внесения внешних данных или предположений.
    # Результат: Ответ должен быть четким и точным, содержать только информацию из предложенного контекста.
    # Ожидается, что ответ будет логически обоснованным и последовательным. Если ответа на вопрос нет в контексте, скажи об этом пользователю
    # Ограничения: Отвечать можно только по контексту."""
    
    # Пример:
    # Контекст: Борщ варится из куриных лапок. Велосипеды марки крути педали имеют три колеса.
    # Вопрос: Когда изобрели утюг?
    # Ответ: Информации нет в контексте.

    # Пример:
    # Контекст: Вася Пупкин изобрёл специальный банкомат в 2000 году. Банкомат нужен для выдачи пончиков.
    # Вопрос: Когда и для чего изобрели специальный банкомат?
    # Ответ: Банкомат был изобретён Васей Пупкиным в 2000 году и он нужен для выдачи пончиков.

    # Пример:
    # Контекст: Автомобильная марка Audi имеет тот же логотип, что и марка ВАЗ. Автомобильная марка ВАЗ славится своими газонокосилками, хоть и производит автомобили.
    # Вопрос: Чем известа автомобильная компания ВАЗ?
    # Ответ: Автомобильная компания ВАЗ производит автомобили и газонокосили. Газонокосилки компании ВАЗ очень известны.
    
    # Контекст: {context}
    # Вопрос: {input}


In [74]:
# system_prompt = (       
#     "Твоя роль: Аналитик специализирующийся на быстром поиске информации в предоставленном контексте."
#     "Краткая инструкция: Анализировать предложенный контекст и отвечать на вопросы, опираясь исключительно на информацию из этого контекста. Если информации в документе нет или не хватает, то так и пиши: информации недостаточно. Всегда отвечай на русском языке."
#     "Твоя цель: Обеспечить структурированный и развёрнутый ответ, полностью основанный на информации из предложенного контекста, без внесения внешних данных или предположений."
#     "Результат: Ответ должен быть четким и точным, содержать только информацию из предложенного контекста."
#     "Ожидается, что ответ будет логически обоснованным и последовательным. Если ответа на вопрос нет в контексте, скажи об этом пользователю"
#     "Ограничения: Отвечать можно только по контексту."
#     "{context}"
#     )

In [75]:
retriever = vectorstore.as_retriever()


In [76]:

document_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt
        )
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [17]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sysprompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


In [ ]:

retrieval_chain_clean = create_retrieval_chain(retriever_clean, question_answer_chain)

In [87]:
retrieval_chain_clean_ens = create_retrieval_chain(ensemble_retriever, question_answer_chain)

In [20]:
from langchain.retrievers import MultiQueryRetriever

In [23]:
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=retriever_d, llm=llm)

In [24]:
retrieval_chain_d = create_retrieval_chain(retriever_from_llm, question_answer_chain)

In [25]:
question1 = "С чего стоит начать  внедрение искусственного интеллекта в HR?"

In [26]:
resp_d = retrieval_chain_d.invoke({'input': question1})
resp_d

{'input': 'С чего стоит начать  внедрение искусственного интеллекта в HR?',
 'context': [Document(metadata={'source': 'ai-hr.pdf', 'page': 15}, page_content='Искусственный интеллект вHR\n16Обучение и повышение квалификации\nОрганизация обучающих курсов по основам \nприменения искусственного интеллекта \nв управлении человеческими ресурсами.\nУчастие во внешних конференциях , курсах \nповышения квалификации и вебинарах, \nпосвященных применению ИИ в HR.\nПартнерство и консультации\nУстановление партнерских отношений \nс компаниями, специализирующимися на ИИ \nи машинном обучении, для получения \nэкспертной поддержки и консультаций.\nПроведение консультаций с внешними \nэкспертами и специалистами по вопросам \nвнедрения ИИ в HR -процессы, чтобы \nразработать стратегию и план действий.\nСоздание команды ИИ в HR\nФормирование специализированной команды \nили группы сотрудников, обладающих знаниями \nи опытом в области искусственного интеллекта \nи анализа данных, внутри HR -подразделения.\

In [32]:
# from langchain.retrievers import MultiQueryRetriever

In [40]:
# retriever_from_llm = MultiQueryRetriever.from_llm(retriever=ensemble_retriever, llm=llm)

In [88]:
resp = retrieval_chain_clean_ens.invoke({'input': question1})
resp

{'input': 'С чего стоит начать  внедрение искусственного интеллекта в HR?',
 'context': [Document(metadata={'source': 'ai-hr.pdf'}, page_content='преимуществ Возможно хорошие результаты позволят руководству увидеть непосредственные практические выгоды внедрения ИИ Прозрачность и открытость Важно поддержание прозрачного и открытого диалога с руководством компании относительно планов по внедрению ИИ в HR обсуждение преимуществ и рисков связанных с этим процессом а также регулярное информирование о прогрессе и достижениях 21 Технологии Доверия 22 Определите стратегию область потенциала готовность организации метрики успеха Внедрение искусственного интеллекта в HR стоит начать с разработки стратегии которая бы включала в себя четкое определение областей и процессов в HR где применение ИИ может существенно повысить эффективность Необходимо ясно сформулировать какие HRзадачи вы хотите решить и каких целей достичь с помощью ИИ Например целью может быть увеличение эффективности процесса подбор

In [28]:
questions = ["С чего стоит начать  внедрение искусственного интеллекта в HR?", 'Что представляет собой искусственный интеллект?','С помощью чего упрощается совместная  работа нескольких пользователей?',
             'На каком этапе работы ИИ помогает автоматизировать и оптимизировать процессы подбора кандидатов?', ' К каким серьезным последствиям для компании может привести утечка  данных о сотрудниках?',
             'Что позволит более эффективно использовать данные для анализа и обучения моделей ИИ?', 'Из-за чего может возникать предвзятость к результатам работы ИИ?', 'Что необходимо проанализировать после завершения пилотного проекта?',
             'Что может стать эффективным способом стимулирования инноваций в организации?', 'Какие навыки потребуются HR-специалистам для работы с ИИ?']

In [90]:
c = 1
for question in questions:
    resp = retrieval_chain.invoke({'input': question})
    respcl = retrieval_chain_clean.invoke({'input': question})
    respens = retrieval_chain_clean_ens.invoke({'input': question})
    print(f'вопрос №{c}')
    print(resp['answer'])
    print('-----------------')
    print(respcl['answer'])
    print('-----------------')
    print(respens['answer'])
    c+=1

вопрос №1
Для начала необходимо определить цели и задачи, которые будут решаться с помощью искусственного интеллекта в HR. Затем следует провести анализ текущих процессов и определить те области, где использование ИИ может быть наиболее эффективным. После этого нужно выбрать подходящие инструменты и технологии ИИ, которые будут соответствовать целям и задачам. Важно также учитывать особенности организации и ее потребности.
-----------------
Для начала внедрения искусственного интеллекта в HR следует определить задачи, которые требуется решить, договориться об образе результата с бизнесом, оценить применимость технологий ИИ для достижения этого образа и сформулировать эффекты, которые принесет решение. Если экономика позволяет, можно смело делать шаг вперед.
-----------------
Для начала внедрения искусственного интеллекта в HR стоит начать с разработки стратегии, которая должна включать четкое определение областей и процессов в HR, где применение ИИ может повысить эффективность. Необход

In [29]:
c = 1
for question in questions:
    resp_d = retrieval_chain_d.invoke({'input': question})
    print(f'вопрос №{c}')
    print(resp_d['answer'])
    c+=1

вопрос №1
С чего стоит начать внедрение искусственного интеллекта в HR?
вопрос №2
Искусственный интеллект (ИИ) представляет собой область компьютерных наук, которая занимается созданием программ и систем, способных имитировать человеческий интеллект и выполнять задачи, требующие когнитивного анализа разнородной информации. Целью развития ИИ является создание систем, способных работать в реальном мире, адаптироваться к новым ситуациям, учиться на опыте и взаимодействовать с людьми и другими системами с помощью естественного языка или других интерфейсов.
вопрос №3
ИИ может упрощать совместную работу нескольких пользователей с помощью напоминаний и подсказок.
вопрос №4
ИИ помогает автоматизировать и оптимизировать процессы подбора кандидатов на этапе анализа резюме, сортировки кандидатов, предварительного скрининга и проведения интервью.
вопрос №5
Утечка данных о сотрудниках может привести к серьезным последствиям для компании, включая ущерб репутации, юридические проблемы и потерю довери

rag fusion

In [91]:
from langchain_core.output_parsers import StrOutputParser

In [104]:
prompt_f = ChatPromptTemplate.from_messages([
    ("system", "Вы полезный помощник на русском языке, который генерирует несколько поисковых запросов на основе одного входного запроса."),
    ("user", "Создайте несколько поисковых запросов, связанных с: {original_query}"),
    ("user", "OUTPUT (4 queries):")
])

In [105]:
generate_queries = (
    prompt_f | llm | StrOutputParser() | (lambda x: x.split("\n"))
)

In [106]:
fus = retriever
fus_cl = retriever_clean

In [176]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results


In [177]:
chainf = generate_queries | retriever.map() | reciprocal_rank_fusion
cgainfclean = generate_queries | retriever_clean.map() | reciprocal_rank_fusion

In [178]:
question1 = "С чего стоит начать  внедрение искусственного интеллекта в HR?"

In [179]:
chainf.invoke({"original_query": question1})

[(Document(metadata={'source': 'ai-hr.pdf', 'page': 4}, page_content='которые могут использоваться в организациях, в частности \nв области управления персоналом.\nПодробно классификацию и историю применения искусственного \nинтеллекта мы рассматривали в нашем исследовании \n«Искусственный интеллект в HR» от 2023 года, с которым можно \nознакомиться по ссылке ( https://tedo.ru/ai -hr).\nИскусственный \nинтеллект вHR\nКейсы российского рынка\nСовместное исследование «Технологий Доверия» и\nKnomary\nВажно понимать, что ИИ –это новая механика, \nинструмент продукта, а не его основа. Подобные \nситуации возникали \nи при предыдущих технологических трендах: продукты \nстановились мобильными и адаптивными, подключали \nданные и аналитику, допускали коллаборацию \nпользователей, но фундаментом продукта оставалась \nфункция решения пользовательской проблемы.\nПервый выпуск исследования\n5'),
  0.06639344262295081),
 (Document(metadata={'source': 'ai-hr.pdf', 'page': 44}, page_content='Если пред

In [121]:
p = cgainfclean.invoke({"original_query": question1})[0]

In [142]:
vop = list(map(lambda x: x[0], data))

(Document(metadata={'source': 'ai-hr.pdf'}, page_content='Использование искусственного интеллекта в HRнаправлении преследует несколько стратегических целей повышение эффективности работы HRфункции улучшение качества принимаемых решений и создание условий для развития персонала Мы уже несколько лет занимаемся приземлением ИИ на наши процессы и первым было все то что человека избавляя от рутины что высвобождает время 39 Что для вас является источником понимания куда двигаться Откуда вы черпаете идеи В 2018 году мы начали с изучения кейсов на платформе Kaggle для того чтобы понять какие задачи имеют решение с помощью технологий ИИ и рейтинговать разработчиков в части компетенций по искусственному интеллекту На кейсах Kaggle мы анализировали результат их решения и его пользу для бизнеса Затем было несколько конференций на которых коллеги из крупных промышленных холдингов показывали свои предиктивные модели для анализа жизненного цикла оборудования'),
 0.03333333333333333)

In [149]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sysprompt),
        ("human", "{original_query}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

retrieval_chainf_clean = create_retrieval_chain(chainf, question_answer_chain)

In [180]:
# resp = retrieval_chainf_clean.invoke({'original_query': question1})